Data Visualization

In [0]:
%sql
USE CATALOG SAMPLES;
SELECT
    hour(tpep_dropoff_datetime) as drop1,
    COUNT(*) AS new
FROM samples.nyctaxi.trips
where pickup_zip in ('10001','10003','10004')
group by 1

drop1,new
12,100
22,130
1,108
13,106
16,117
6,36
3,53
20,169
5,24
19,164


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Delta live table

In [0]:
%sql
create or refresh streaming table taxi_raw_records
(constraint valid expect(trip_distance>0.0)on violation drop row)
as
select * 
from 
STREAM(samples.nyctaxi.trips)

Name,Type
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
trip_distance,double
fare_amount,double
pickup_zip,int
dropoff_zip,int


Writing data into delta tables
Creating dataframes from datasets and reading them in different formats


In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Delta").getOrCreate()

In [0]:
s1=spark.read.option("inferschema",True).option("header",True).csv("/FileStore/tables/new1.txt")
s1.write.format("delta").mode("overwrite").save("/FileStore/tables/delta_train/")  


In [0]:
display(dbutils.fs.ls("/FileStore/tables/delta_train/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/delta_train/_delta_log/,_delta_log/,0,1733830331000
dbfs:/FileStore/tables/delta_train/part-00000-9612b4b2-1b8d-41a2-826e-10dbcd6ab9cd-c000.snappy.parquet,part-00000-9612b4b2-1b8d-41a2-826e-10dbcd6ab9cd-c000.snappy.parquet,3872,1733830332000


In [0]:
data=spark.read.format("delta").load("dbfs:/user/hive/warehouse/export") #Reading table in delta format
data.show(10)

+---+---------+----------+---------+------+-------------------+-----------+------+
| id|firstName|middleName| lastName|gender|          birthDate|        ssn|salary|
+---+---------+----------+---------+------+-------------------+-----------+------+
|  2|       An|     Amira|   Cowper|     F|1992-02-08 05:00:00|978-97-8086| 40203|
| 19|   Adelle|   Kathyrn|Grigoriev|     F|1978-11-14 05:00:00|923-23-5984| 60600|
| 40|  Alethea|     Dolly|Brickdale|     F|1960-06-26 04:00:00|908-46-4236| 89151|
| 61|   Adelia|      Gita|   Vassel|     F|1990-10-24 04:00:00|947-17-3832| 63563|
| 63|Anjanette|    Clelia|    Hicks|     F|1973-10-26 04:00:00|981-32-5795| 61310|
| 75|    Alica|  Elfrieda| Mousdall|     F|1953-12-21 05:00:00|954-16-6401| 96490|
| 90|    Alice|   Marisol|   Novill|     F|1992-05-21 04:00:00|924-10-7563| 34053|
|100| Annalisa|      Nova|  Patesel|     F|1974-01-25 04:00:00|922-68-2542| 66481|
|105|    Annie|    Jeanne|  Beecker|     F|1961-12-09 05:00:00|903-15-7880| 63493|
|110

In [0]:
from pyspark.sql.functions import *
dfresult=data.select("*").filter(col("lastName").endswith("r"))
display(dfresult)

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
2,An,Amira,Cowper,F,1992-02-08T05:00:00Z,978-97-8086,40203
105,Annie,Jeanne,Beecker,F,1961-12-09T05:00:00Z,903-15-7880,63493
293,Alida,Peggy,Bracher,F,1972-09-18T04:00:00Z,953-82-2303,107303
296,Ann,Vinita,Roscher,F,1986-05-31T04:00:00Z,936-40-3049,36909
348,Andra,Ami,Player,F,1984-09-10T04:00:00Z,986-41-7846,97363
640,Angelic,Rossana,Raiker,F,1969-06-07T04:00:00Z,985-74-9894,30965
646,Alecia,Latrice,Dener,F,1973-04-02T05:00:00Z,926-24-5548,64624
835,Antoinette,Era,Warriner,F,1982-07-08T04:00:00Z,954-43-1908,69049
2,An,Amira,Cowper,F,1992-02-08T05:00:00Z,978-97-8086,40203
105,Annie,Jeanne,Beecker,F,1961-12-09T05:00:00Z,903-15-7880,63493


Dbutils

In [0]:
%fs
ls /databricks-datasets     #listing all the datasets in filestore


path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/,COVID/,0,1733830997723
dbfs:/databricks-datasets/README.md,README.md,976,1532502320000
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0,1733830997723
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359,1516124912000
dbfs:/databricks-datasets/adult/,adult/,0,1733830997723
dbfs:/databricks-datasets/airlines/,airlines/,0,1733830997723
dbfs:/databricks-datasets/amazon/,amazon/,0,1733830997723
dbfs:/databricks-datasets/asa/,asa/,0,1733830997723
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0,1733830997723
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0,1733830997723


In [0]:
dbutils.secrets.listScopes()
#Listing the secret scope and Used to securely store and retrieve secrets, like credentials and API keys.

[]

In [0]:
dbutils.widgets.text("input", "default_value", "Enter a value")#Creating a text widget
dbutils.widgets.get("input")

'default_value'

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/flights/"))

path,name,size,modificationTime
dbfs:/databricks-datasets/flights/README.md,README.md,412,1516326253000
dbfs:/databricks-datasets/flights/airport-codes-na.txt,airport-codes-na.txt,11411,1516326394000
dbfs:/databricks-datasets/flights/departuredelays.csv,departuredelays.csv,33396236,1516326403000


In [0]:
for foldername in dbutils.fs.ls("databricks-datasets/"):
    print(foldername.name)

COVID/
README.md
Rdatasets/
SPARK_README.md
adult/
airlines/
amazon/
asa/
atlas_higgs/
bikeSharing/
cctvVideos/
credit-card-fraud/
cs100/
cs110x/
cs190/
data.gov/
definitive-guide/
delta-sharing/
flights/
flower_photos/
flowers/
genomics/
hail/
identifying-campaign-effectiveness/
iot/
iot-stream/
learning-spark/
learning-spark-v2/
lending-club-loan-stats/
med-images/
media/
mnist-digits/
news20.binary/
nyctaxi/
nyctaxi-with-zipcodes/
online_retail/
overlap-join/
power-plant/
retail-org/
rwe/
sai-summit-2019-sf/
sample_logs/
samples/
sfo_customer_survey/
sms_spam_collection/
songs/
structured-streaming/
timeseries/
tpch/
travel_recommendations_realtime/
warmup/
weather/
wiki/
wikipedia-datasets/
wine-quality/


In [0]:
dbutils.notebook.run("/Workspace/Users/azuser2362_mml.local@techademy.com/new",10)